# ICLR 2026 Acceptance Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leippold/HAI-Frontier/blob/main/acceptance_analysis_notebook.ipynb)

## Analysis of AI Content Effects on Acceptance Decisions

**ICLR 2026 decisions are now available!** This notebook analyzes whether AI-generated content in papers affects acceptance decisions.

### Tests Implemented:

#### Part A: AI Content → Acceptance
1. **AI → Acceptance | Scores** - Probit/Logit regression testing if AI content predicts acceptance conditional on review scores
2. **Threshold Discontinuity** - RDD-style analysis near the decision margin (tiebreaker effect)
3. **Presentation Tier Analysis** - Ordered probit for tier assignment among accepted papers
4. **Author Reputation × Acceptance** - Does reputation insulate from AI penalty?

#### Part B: Echo Chamber Analysis (AI Reviews)
5. **AI Reviews × AI Papers** - Do AI-assisted reviews rate AI papers differently?
6. **Acceptance | AI Review Ratings** - Does acceptance depend on AI content conditional on AI reviewer scores?

### Key Research Questions:
- Does AI content predict acceptance **conditional on scores**? (discrimination beyond the score channel)
- Is AI content used as a **tiebreaker** near the acceptance margin?
- Are high-AI papers relegated to **lower presentation tiers**?
- Does **author reputation** protect against the AI penalty?
- Do **AI reviewers** systematically favor or penalize **AI papers**? (echo chamber effect)

In [ ]:
#@title 1. Load GitHub Token from Colab Secrets { display-mode: "form" }
#@markdown Add your GitHub token to Colab Secrets (key icon in left sidebar) with name `GITHUB_TOKEN`

from google.colab import userdata

try:
    GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
    print("✓ GitHub token loaded from Colab Secrets")
except:
    GITHUB_TOKEN = None
    print("⚠️  No GitHub token found - clone may fail for private repos")

In [ ]:
#@title 2. Configuration { display-mode: "form" }

#@markdown ### Data Location
#@markdown Path to your data folder in Google Drive:
DATA_PATH = "/content/drive/MyDrive/HAI_Data"  #@param {type:"string"}

#@markdown ---
#@markdown ### GitHub Configuration
GITHUB_USER = "leippold"  #@param {type:"string"}
REPO_NAME = "HAI-Frontier"  #@param {type:"string"}

In [ ]:
#@title 3. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive mounted")

In [ ]:
#@title 4. Clone Repository from GitHub
import os

%cd /content

# Clean up any existing clone
!rm -rf /content/{REPO_NAME}

# Clone the repo (with token if available)
if GITHUB_TOKEN:
    !git clone https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git
else:
    !git clone https://github.com/{GITHUB_USER}/{REPO_NAME}.git

# Change to repo directory
%cd /content/{REPO_NAME}

print(f"\n✓ Repository cloned to: /content/{REPO_NAME}")

In [ ]:
#@title 5. Install Dependencies & Setup Python Path
!pip install -q pandas numpy matplotlib seaborn scipy statsmodels requests openreview-py

import sys
import os
REPO_PATH = f"/content/{REPO_NAME}"
sys.path.insert(0, f"{REPO_PATH}/iclr_analysis")

# Create output directory
os.makedirs('last_results', exist_ok=True)

print("✓ Dependencies installed (including openreview-py)")
print(f"✓ Python path configured")
print(f"✓ Working directory: {os.getcwd()}")

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

In [ ]:
# Import custom modules
import sys
sys.path.insert(0, 'iclr_analysis')

from src.openreview_api import (
    fetch_iclr_decisions,
    merge_acceptance_data,
    load_or_fetch_decisions,
    check_decision_availability,
    extract_forum_id
)

from analysis.acceptance_analysis import (
    run_acceptance_analysis,
    ai_acceptance_probit,
    threshold_discontinuity,
    optimal_bandwidth_selection,
    presentation_tier_analysis,
    reputation_acceptance_interaction,
    selection_bounds_analysis,
    score_acceptance_residual,
    prepare_acceptance_data,
    ai_acceptance_by_category
)

# Echo chamber analysis for AI reviews
from analysis.echo_chamber import (
    run_echo_chamber_analysis,
    compute_interaction_effect,
    test_interaction_comprehensive
)

from src.data_loading import load_data, clean_ai_percentage, prepare_echo_chamber_data

## 1. Load Data

First, we load the existing submissions data and check if acceptance decisions are available.

In [ ]:
# Load submissions data
submissions_path = 'data/iclr_submissions_enriched.csv'
submissions_df = pd.read_csv(submissions_path)

print(f"Loaded {len(submissions_df):,} submissions")
print(f"\nColumns: {list(submissions_df.columns)}")

In [ ]:
# Check if acceptance data already exists
if 'accepted' in submissions_df.columns:
    print("Acceptance data already present!")
    print(f"Acceptance rate: {submissions_df['accepted'].mean():.1%}")
else:
    print("No acceptance data found - will need to fetch from OpenReview")

#@title 2. Fetch Acceptance Data from OpenReview

#@markdown If acceptance decisions are not in the data, we fetch them from OpenReview.
#@markdown 
#@markdown **Note**: This requires ICLR to have released decisions. Check availability first.
#@markdown 
#@markdown ### Caching Strategy
#@markdown Fetched acceptance data is cached in Google Drive (`DATA_PATH`) to avoid redundant API calls.
#@markdown Set `USE_CACHE = True` to load from cache if available.

USE_CACHE = True  #@param {type:"boolean"}
ICLR_YEAR = 2026  #@param {type:"integer"}

import os

# Cache file path in Google Drive
ACCEPTANCE_CACHE_FILE = f"{DATA_PATH}/iclr{ICLR_YEAR}_acceptance_data.csv"

print(f"Cache file: {ACCEPTANCE_CACHE_FILE}")
print(f"Use cache: {USE_CACHE}")

In [ ]:
#@title Fetch ICLR 2026 Accepted Papers from OpenReview (with Caching)
import openreview
import os

venue_id = f'ICLR.cc/{ICLR_YEAR}/Conference'
accepted_ids = set()
accepted_titles = {}

# Check if cached data exists in Google Drive
if USE_CACHE and os.path.exists(ACCEPTANCE_CACHE_FILE):
    print(f"✓ Loading cached acceptance data from Google Drive...")
    cached_df = pd.read_csv(ACCEPTANCE_CACHE_FILE)
    accepted_ids = set(cached_df['forum_id'].dropna().tolist())
    
    # Reconstruct title mapping if available
    if 'title' in cached_df.columns:
        for _, row in cached_df.iterrows():
            if pd.notna(row.get('title')):
                accepted_titles[row['title'].lower().strip()] = row.get('forum_id', '')
    
    print(f"✓ Loaded {len(accepted_ids):,} accepted paper IDs from cache")
    print(f"  With titles: {len(accepted_titles):,}")
    
else:
    print(f"Connecting to OpenReview API...")
    client = openreview.api.OpenReviewClient(baseurl='https://api2.openreview.net')

    print(f"Fetching accepted papers for {venue_id}...")
    accepted_papers = list(client.get_all_notes(content={'venueid': venue_id}))

    print(f"\n✓ Found {len(accepted_papers):,} accepted papers")

    # Extract accepted paper IDs and titles
    accepted_data = []
    for note in accepted_papers:
        accepted_ids.add(note.id)
        title = note.content.get('title', {}).get('value', '')
        if title:
            accepted_titles[title.lower().strip()] = note.id
        accepted_data.append({
            'forum_id': note.id,
            'title': title,
            'venue_id': venue_id
        })

    print(f"  Unique IDs: {len(accepted_ids)}")
    print(f"  With titles: {len(accepted_titles)}")
    
    # Save to Google Drive cache
    print(f"\n📁 Saving acceptance data to Google Drive cache...")
    accepted_cache_df = pd.DataFrame(accepted_data)
    os.makedirs(os.path.dirname(ACCEPTANCE_CACHE_FILE), exist_ok=True)
    accepted_cache_df.to_csv(ACCEPTANCE_CACHE_FILE, index=False)
    print(f"✓ Saved to: {ACCEPTANCE_CACHE_FILE}")

In [ ]:
#@title Match Accepted Papers with AI Content Data
from src.openreview_api import extract_forum_id

# Extract forum IDs from our submissions data
submissions_df['forum_id'] = submissions_df['openreview_url'].apply(extract_forum_id)

# Match by forum_id first
submissions_df['accepted'] = submissions_df['forum_id'].isin(accepted_ids).astype(int)

# For papers without forum_id match, try title matching
no_match = submissions_df['accepted'] == 0
if no_match.any():
    submissions_df['_title_norm'] = submissions_df['title'].str.lower().str.strip()
    title_matches = submissions_df['_title_norm'].isin(accepted_titles.keys())
    submissions_df.loc[title_matches & no_match, 'accepted'] = 1
    submissions_df = submissions_df.drop(columns=['_title_norm'])

# Create df_with_decisions for downstream analysis
df_with_decisions = submissions_df.copy()

# Summary
n_accepted = df_with_decisions['accepted'].sum()
n_total = len(df_with_decisions)
acceptance_rate = n_accepted / n_total

print("=" * 60)
print("MATCHING RESULTS")
print("=" * 60)
print(f"Total submissions in data: {n_total:,}")
print(f"Matched as accepted: {n_accepted:,}")
print(f"Acceptance rate: {acceptance_rate:.1%}")
print(f"\nExpected from OpenReview: {len(accepted_papers):,} accepted")

if n_accepted != len(accepted_papers):
    print(f"\n⚠️  Difference: {len(accepted_papers) - n_accepted} papers")
    print("   (Some accepted papers may not be in our submission data)")

print(f"\n✓ 'accepted' column added to dataframe")

In [ ]:
#@title Verify Data Quality
from src.data_loading import clean_ai_percentage

# Clean AI percentage column
df_with_decisions = clean_ai_percentage(df_with_decisions)

print("=" * 60)
print("DATA QUALITY CHECK")
print("=" * 60)

print(f"\nSample of accepted papers:")
accepted_sample = df_with_decisions[df_with_decisions['accepted'] == 1][['title', 'avg_rating', 'ai_percentage']].head(5)
print(accepted_sample.to_string())

print(f"\nSample of rejected papers:")
rejected_sample = df_with_decisions[df_with_decisions['accepted'] == 0][['title', 'avg_rating', 'ai_percentage']].head(5)
print(rejected_sample.to_string())

print(f"\nAI percentage distribution:")
print(df_with_decisions['ai_percentage'].describe())

print(f"\nMissing values:")
print(f"  avg_rating: {df_with_decisions['avg_rating'].isna().sum()}")
print(f"  ai_percentage: {df_with_decisions['ai_percentage'].isna().sum()}")

## 3. Exploratory Data Analysis

In [ ]:
# Summary statistics
if 'accepted' in df_with_decisions.columns:
    print("=" * 60)
    print("DATA SUMMARY")
    print("=" * 60)
    
    print(f"\nTotal papers: {len(df_with_decisions):,}")
    print(f"Acceptance rate: {df_with_decisions['accepted'].mean():.1%}")
    print(f"Mean AI percentage: {df_with_decisions['ai_percentage'].mean():.1f}%")
    print(f"Mean avg_rating: {df_with_decisions['avg_rating'].mean():.2f}")
    
    if 'tier_name' in df_with_decisions.columns:
        print(f"\nTier distribution:")
        print(df_with_decisions['tier_name'].value_counts())

In [ ]:
# Visualize acceptance by AI content and rating
if 'accepted' in df_with_decisions.columns:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 1. Rating distribution by acceptance
    ax1 = axes[0]
    df_with_decisions.boxplot(column='avg_rating', by='accepted', ax=ax1)
    ax1.set_xlabel('Accepted')
    ax1.set_ylabel('Average Rating')
    ax1.set_title('Rating by Acceptance')
    plt.suptitle('')
    
    # 2. AI content by acceptance
    ax2 = axes[1]
    df_with_decisions.boxplot(column='ai_percentage', by='accepted', ax=ax2)
    ax2.set_xlabel('Accepted')
    ax2.set_ylabel('AI Percentage')
    ax2.set_title('AI Content by Acceptance')
    
    # 3. Acceptance rate by AI quartile
    ax3 = axes[2]
    df_temp = df_with_decisions.copy()
    df_temp['ai_quartile'] = pd.qcut(df_temp['ai_percentage'], 4, labels=['Q1 (Low)', 'Q2', 'Q3', 'Q4 (High)'])
    acc_by_quartile = df_temp.groupby('ai_quartile')['accepted'].mean()
    acc_by_quartile.plot(kind='bar', ax=ax3, color='steelblue', edgecolor='black')
    ax3.set_xlabel('AI Content Quartile')
    ax3.set_ylabel('Acceptance Rate')
    ax3.set_title('Acceptance Rate by AI Quartile')
    ax3.tick_params(axis='x', rotation=0)
    ax3.axhline(y=df_with_decisions['accepted'].mean(), color='red', linestyle='--', label='Overall')
    ax3.legend()
    
    plt.tight_layout()
    plt.show()

## 4. Run Comprehensive Acceptance Analysis

Run all statistical tests at once.

In [ ]:
# Run all acceptance analysis tests
if 'accepted' in df_with_decisions.columns:
    results = run_acceptance_analysis(
        df_with_decisions,
        controls=['first_author_h_index', 'international_collaboration'],  # Optional controls
        run_all=True,
        verbose=True
    )
else:
    print("No acceptance data available. Please fetch decisions first.")

## 5. Individual Tests (Detailed Results)

Run individual tests for more detailed examination.

### Test 1: AI → Acceptance | Scores (Probit Regression)

In [ ]:
# Test 1: AI → Acceptance conditional on scores
if 'accepted' in df_with_decisions.columns:
    probit_results = ai_acceptance_probit(
        df_with_decisions,
        controls=['first_author_h_index'],
        verbose=True
    )

In [ ]:
# Unadjusted comparison by AI category
if 'accepted' in df_with_decisions.columns:
    category_results = ai_acceptance_by_category(df_with_decisions, verbose=True)

### Test 2: Threshold Discontinuity (Tiebreaker Analysis)

In [ ]:
# Test 2: Threshold discontinuity
if 'accepted' in df_with_decisions.columns:
    # First, let's look at the score distribution to find a good margin
    print("Score distribution around decision threshold:")
    print(df_with_decisions['avg_rating'].describe())
    
    # Acceptance rate by score bin
    df_temp = df_with_decisions.copy()
    df_temp['score_bin'] = pd.cut(df_temp['avg_rating'], bins=20)
    acc_by_score = df_temp.groupby('score_bin').agg({
        'accepted': ['mean', 'count']
    }).round(3)
    print("\nAcceptance rate by score bin:")
    print(acc_by_score)

In [ ]:
# Run threshold discontinuity analysis
if 'accepted' in df_with_decisions.columns:
    # Find approximate decision threshold (where acceptance ~ 50%)
    disc_results = threshold_discontinuity(
        df_with_decisions,
        margin_lower=4.8,  # Adjust based on your data
        margin_upper=5.2,
        verbose=True
    )

In [ ]:
# Bandwidth sensitivity analysis
if 'accepted' in df_with_decisions.columns:
    bandwidth_results = optimal_bandwidth_selection(
        df_with_decisions,
        verbose=True
    )

### Test 3: Presentation Tier Analysis

In [ ]:
# Test 3: Presentation tier analysis (among accepted papers)
if 'accepted' in df_with_decisions.columns and ('tier' in df_with_decisions.columns or 'tier_name' in df_with_decisions.columns):
    tier_results = presentation_tier_analysis(
        df_with_decisions,
        verbose=True
    )
else:
    print("Tier data not available")

### Test 4: Author Reputation Interaction

In [ ]:
# Test 4: Reputation × AI interaction
if 'accepted' in df_with_decisions.columns and 'first_author_h_index' in df_with_decisions.columns:
    # Check data availability
    n_with_hindex = df_with_decisions['first_author_h_index'].notna().sum()
    print(f"Papers with h-index data: {n_with_hindex}")
    
    if n_with_hindex >= 100:
        rep_results = reputation_acceptance_interaction(
            df_with_decisions,
            reputation_col='first_author_h_index',
            verbose=True
        )
    else:
        print("Insufficient h-index data for this analysis")
else:
    print("Required columns not available")

### Test 5: Selection Bounds Analysis

In [ ]:
# Test 5: Selection bounds
if 'first_author_h_index' in df_with_decisions.columns:
    bounds_results = selection_bounds_analysis(
        df_with_decisions,
        observed_col='first_author_h_index',
        verbose=True
    )

### Score-Acceptance Residual Analysis

In [ ]:
# Residual analysis
if 'accepted' in df_with_decisions.columns:
    residual_results = score_acceptance_residual(
        df_with_decisions,
        verbose=True
    )

## Part B: Echo Chamber Analysis (AI Reviews Rating AI Papers)

This section tests whether AI-assisted reviews systematically rate AI-written papers differently.

**Key Question**: Do AI reviewers give higher/lower ratings to AI papers compared to human reviewers?

The interaction effect captures the "echo chamber" hypothesis:
- Positive interaction → AI reviewers favor AI papers
- Negative interaction → AI reviewers penalize AI papers

In [ ]:
# Prepare data for echo chamber analysis
# This requires review-level data with AI percentage for each review
# If only paper-level avg_rating is available, we analyze AI content effects on ratings

if 'accepted' in df_with_decisions.columns:
    # Create AI paper indicator
    df_echo = df_with_decisions.copy()
    df_echo = clean_ai_percentage(df_echo)
    
    # Define AI paper threshold (papers with high AI content)
    AI_PAPER_THRESHOLD = 75  # Papers with >=75% AI content are "AI papers"
    HUMAN_PAPER_THRESHOLD = 25  # Papers with <=25% AI content are "human papers"
    
    df_echo['paper_type'] = pd.cut(
        df_echo['ai_percentage'],
        bins=[-1, HUMAN_PAPER_THRESHOLD, AI_PAPER_THRESHOLD, 101],
        labels=['Human Paper', 'Mixed', 'AI Paper']
    )
    
    print("Paper Type Distribution:")
    print(df_echo['paper_type'].value_counts())
    print(f"\nAI Paper threshold: >= {AI_PAPER_THRESHOLD}%")
    print(f"Human Paper threshold: <= {HUMAN_PAPER_THRESHOLD}%")

In [ ]:
# Compare acceptance rates: AI Papers vs Human Papers (with Effect Sizes)
if 'accepted' in df_with_decisions.columns:
    from scipy import stats
    from src.stats_utils import cohens_d, hedges_g, cliffs_delta, bootstrap_diff_ci
    
    print("=" * 70)
    print("ACCEPTANCE RATES BY PAPER TYPE (with Effect Sizes)")
    print("=" * 70)
    
    # Exclude mixed papers for clean comparison
    df_clean = df_echo[df_echo['paper_type'].isin(['Human Paper', 'AI Paper'])]
    
    acc_by_type = df_clean.groupby('paper_type').agg({
        'accepted': ['mean', 'sum', 'count'],
        'avg_rating': ['mean', 'std']
    }).round(3)
    acc_by_type.columns = ['Acceptance Rate', 'N Accepted', 'N Total', 'Mean Rating', 'Std Rating']
    print(acc_by_type)
    
    # Get acceptance arrays for each group
    ai_papers = df_clean[df_clean['paper_type'] == 'AI Paper']['accepted'].values
    human_papers = df_clean[df_clean['paper_type'] == 'Human Paper']['accepted'].values
    
    # Chi-square test for acceptance rates
    contingency = pd.crosstab(df_clean['paper_type'], df_clean['accepted'])
    chi2, p_chi2, dof, expected = stats.chi2_contingency(contingency)
    
    # Cramér's V (effect size for chi-square)
    n = contingency.sum().sum()
    min_dim = min(contingency.shape) - 1
    cramers_v = np.sqrt(chi2 / (n * min_dim)) if min_dim > 0 else 0
    
    print(f"\n*** Chi-square test for independence ***")
    print(f"Chi2: {chi2:.3f}, p-value: {p_chi2:.4e}")
    print(f"Cramér's V (effect size): {cramers_v:.4f}")
    print(f"Significant difference: {'YES' if p_chi2 < 0.05 else 'NO'}")
    
    # Effect sizes for the acceptance rate difference
    print(f"\n*** Effect Sizes ***")
    
    # Cohen's d
    d = cohens_d(ai_papers, human_papers)
    print(f"Cohen's d: {d:.4f}")
    
    # Hedges' g (bias-corrected)
    g = hedges_g(ai_papers, human_papers)
    print(f"Hedges' g (bias-corrected): {g:.4f}")
    
    # Cliff's delta (non-parametric)
    delta = cliffs_delta(ai_papers, human_papers)
    print(f"Cliff's Delta: {delta:.4f}")
    
    # Bootstrap confidence interval for the difference
    boot_ci = bootstrap_diff_ci(ai_papers, human_papers, n_bootstrap=5000)
    print(f"\n*** Bootstrap CI for Acceptance Rate Difference ***")
    print(f"Difference (AI - Human): {boot_ci['diff']:.4f}")
    print(f"95% CI: [{boot_ci['ci_lower']:.4f}, {boot_ci['ci_upper']:.4f}]")
    print(f"Bootstrap SE: {boot_ci['se']:.4f}")
    
    # Interpretation
    print(f"\n*** Interpretation ***")
    if abs(d) < 0.2:
        size_label = "negligible"
    elif abs(d) < 0.5:
        size_label = "small"
    elif abs(d) < 0.8:
        size_label = "medium"
    else:
        size_label = "large"
    print(f"Effect size is {size_label} (|d| = {abs(d):.3f})")

### Acceptance Conditional on Review Ratings

Does AI content predict acceptance **beyond** what review scores would predict?

In [ ]:
# Acceptance conditional on rating - Does AI content matter beyond scores?
if 'accepted' in df_with_decisions.columns:
    from scipy import stats
    import statsmodels.api as sm
    from statsmodels.discrete.discrete_model import Probit, Logit
    
    print("=" * 70)
    print("ACCEPTANCE CONDITIONAL ON REVIEW RATINGS")
    print("=" * 70)
    
    # Prepare data
    df_model = df_echo.dropna(subset=['accepted', 'avg_rating', 'ai_percentage'])
    
    # Create design matrix
    X = df_model[['avg_rating', 'ai_percentage']].copy()
    X = sm.add_constant(X)
    y = df_model['accepted']
    
    print(f"\nSample size: {len(df_model):,}")
    
    # Fit probit model
    try:
        probit_model = Probit(y, X)
        probit_results = probit_model.fit(disp=0)
        
        print("\n*** Probit Model: Acceptance ~ Rating + AI% ***")
        print(probit_results.summary2().tables[1].to_string())
        
        # Key coefficient: AI percentage
        ai_coef = probit_results.params['ai_percentage']
        ai_pval = probit_results.pvalues['ai_percentage']
        ai_se = probit_results.bse['ai_percentage']
        
        print(f"\n*** KEY RESULT: AI Percentage Effect ***")
        print(f"Coefficient: {ai_coef:.5f}")
        print(f"Std Error: {ai_se:.5f}")
        print(f"p-value: {ai_pval:.4e}")
        print(f"Significant at 5%: {'YES' if ai_pval < 0.05 else 'NO'}")
        
        # Interpretation
        if ai_coef < 0 and ai_pval < 0.05:
            print("\n--> AI content NEGATIVELY affects acceptance beyond review scores")
        elif ai_coef > 0 and ai_pval < 0.05:
            print("\n--> AI content POSITIVELY affects acceptance beyond review scores")
        else:
            print("\n--> AI content effect is NOT significant conditional on scores")
            print("    (Any AI penalty operates through the score channel)")
            
    except Exception as e:
        print(f"Model fitting failed: {e}")

In [ ]:
# Stratified analysis: Acceptance by rating bins for AI vs Human papers
if 'accepted' in df_with_decisions.columns:
    print("=" * 70)
    print("STRATIFIED ANALYSIS: ACCEPTANCE BY RATING BINS")
    print("=" * 70)
    
    df_strat = df_clean.copy()
    df_strat['rating_bin'] = pd.qcut(df_strat['avg_rating'], q=5, labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])
    
    # Acceptance by rating bin and paper type
    strat_results = df_strat.groupby(['rating_bin', 'paper_type'])['accepted'].agg(['mean', 'count']).round(3)
    print("\nAcceptance Rate by Rating Bin and Paper Type:")
    print(strat_results.unstack())
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Acceptance rate by AI percentage (continuous)
    ax1 = axes[0]
    df_plot = df_echo.copy()
    df_plot['ai_decile'] = pd.qcut(df_plot['ai_percentage'], 10, labels=False, duplicates='drop')
    acc_by_ai = df_plot.groupby('ai_decile').agg({
        'ai_percentage': 'mean',
        'accepted': ['mean', 'sem']
    })
    acc_by_ai.columns = ['ai_mean', 'acc_rate', 'acc_se']
    
    ax1.errorbar(acc_by_ai['ai_mean'], acc_by_ai['acc_rate'], 
                 yerr=1.96*acc_by_ai['acc_se'], fmt='o-', capsize=3, color='steelblue')
    ax1.axhline(y=df_echo['accepted'].mean(), color='red', linestyle='--', alpha=0.5, label='Overall Rate')
    ax1.set_xlabel('AI Percentage')
    ax1.set_ylabel('Acceptance Rate')
    ax1.set_title('ICLR 2026: Acceptance Rate by AI Content Level')
    ax1.legend()
    
    # Plot 2: Acceptance rate by paper type within rating strata
    ax2 = axes[1]
    pivot = df_strat.pivot_table(values='accepted', index='rating_bin', columns='paper_type', aggfunc='mean')
    pivot.plot(kind='bar', ax=ax2, color=['steelblue', 'coral'], edgecolor='black')
    ax2.set_xlabel('Rating Bin')
    ax2.set_ylabel('Acceptance Rate')
    ax2.set_title('ICLR 2026: Acceptance by Rating Bin - AI vs Human Papers')
    ax2.tick_params(axis='x', rotation=45)
    ax2.legend(title='Paper Type')
    
    plt.tight_layout()
    plt.savefig('last_results/iclr2026_acceptance_by_ai_content.png', dpi=150, bbox_inches='tight')
    plt.show()

## 8. ICLR 2026 Interpretation and Conclusions

### Key Findings:

1. **AI → Acceptance | Scores**: [Result will appear after running]
   - If significant negative: Area chairs penalize AI content beyond reviewer scores
   - If not significant: Any AI penalty operates through the score channel

2. **Threshold Discontinuity**: [Result will appear after running]
   - If significant: AI content is used as a tiebreaker at the margin
   - If not significant: AI content doesn't affect marginal decisions

3. **Presentation Tier**: [Result will appear after running]
   - If significant negative: High-AI papers get relegated to poster tier
   - Conditional on acceptance, so interpretation differs from rejection

4. **Reputation × AI**: [Result will appear after running]
   - If significant positive: Senior authors are protected from AI penalty
   - Selection bias caveat applies

5. **Echo Chamber (AI Reviews)**: [Result will appear after running]
   - Positive interaction: AI reviewers favor AI papers
   - Negative interaction: AI reviewers penalize AI papers
   - This tests whether the review process itself is contaminated

In [ ]:
# Save enriched ICLR 2026 data with acceptance info
if 'accepted' in df_with_decisions.columns:
    # Save to repo directory (for git tracking)
    repo_output_path = f'data/iclr{ICLR_YEAR}_submissions_with_acceptance.csv'
    df_with_decisions.to_csv(repo_output_path, index=False)
    print(f"Saved to repo: {repo_output_path}")
    
    # Also save to Google Drive (for persistence across sessions)
    gdrive_output_path = f'{DATA_PATH}/iclr{ICLR_YEAR}_submissions_with_acceptance.csv'
    os.makedirs(os.path.dirname(gdrive_output_path), exist_ok=True)
    df_with_decisions.to_csv(gdrive_output_path, index=False)
    print(f"Saved to Google Drive: {gdrive_output_path}")
    
    # Summary statistics
    print(f"\n{'='*60}")
    print(f"ICLR {ICLR_YEAR} FINAL SUMMARY")
    print(f"{'='*60}")
    print(f"Total submissions: {len(df_with_decisions):,}")
    print(f"Accepted: {df_with_decisions['accepted'].sum():,} ({df_with_decisions['accepted'].mean():.1%})")
    print(f"Mean AI percentage: {df_with_decisions['ai_percentage'].mean():.1f}%")
    print(f"Mean avg_rating: {df_with_decisions['avg_rating'].mean():.2f}")
    
    if 'tier_name' in df_with_decisions.columns:
        print(f"\nPresentation Tier Distribution:")
        print(df_with_decisions[df_with_decisions['accepted']==1]['tier_name'].value_counts())

## 6. Visualizations

In [ ]:
# Visualization: Acceptance by AI content and rating
if 'accepted' in df_with_decisions.columns:
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. Heatmap of acceptance rate by rating × AI content
    ax1 = axes[0, 0]
    df_temp = df_with_decisions.copy()
    df_temp['rating_bin'] = pd.cut(df_temp['avg_rating'], bins=5)
    df_temp['ai_bin'] = pd.cut(df_temp['ai_percentage'], bins=5)
    pivot = df_temp.pivot_table(values='accepted', index='ai_bin', columns='rating_bin', aggfunc='mean')
    sns.heatmap(pivot, annot=True, fmt='.2f', cmap='RdYlGn', ax=ax1)
    ax1.set_title('Acceptance Rate by Rating and AI Content')
    ax1.set_xlabel('Average Rating')
    ax1.set_ylabel('AI Percentage')
    
    # 2. Acceptance rate vs AI percentage (binned)
    ax2 = axes[0, 1]
    df_temp['ai_decile'] = pd.qcut(df_temp['ai_percentage'], 10, labels=False, duplicates='drop')
    acc_by_ai = df_temp.groupby('ai_decile').agg({
        'ai_percentage': 'mean',
        'accepted': ['mean', 'sem', 'count']
    })
    acc_by_ai.columns = ['ai_mean', 'acc_rate', 'acc_se', 'n']
    
    ax2.errorbar(acc_by_ai['ai_mean'], acc_by_ai['acc_rate'], 
                 yerr=1.96*acc_by_ai['acc_se'], fmt='o-', capsize=3)
    ax2.set_xlabel('AI Percentage')
    ax2.set_ylabel('Acceptance Rate')
    ax2.set_title('Acceptance Rate by AI Content')
    ax2.axhline(y=df_with_decisions['accepted'].mean(), color='red', linestyle='--', alpha=0.5)
    
    # 3. Regression discontinuity plot
    ax3 = axes[1, 0]
    margin_df = df_with_decisions[
        (df_with_decisions['avg_rating'] >= 4.5) & 
        (df_with_decisions['avg_rating'] <= 5.5)
    ]
    ai_median = margin_df['ai_percentage'].median()
    
    high_ai = margin_df[margin_df['ai_percentage'] >= ai_median]
    low_ai = margin_df[margin_df['ai_percentage'] < ai_median]
    
    for label, subset, color in [('High AI', high_ai, 'red'), ('Low AI', low_ai, 'green')]:
        subset_sorted = subset.sort_values('avg_rating')
        rolling_acc = subset_sorted['accepted'].rolling(20, min_periods=10).mean()
        ax3.plot(subset_sorted['avg_rating'], rolling_acc, label=label, color=color, alpha=0.7)
    
    ax3.axvline(x=5.0, color='black', linestyle='--', alpha=0.5, label='Threshold')
    ax3.set_xlabel('Average Rating')
    ax3.set_ylabel('Acceptance Rate (Rolling Mean)')
    ax3.set_title('Acceptance Near Margin by AI Content')
    ax3.legend()
    
    # 4. Tier distribution by AI content (if available)
    ax4 = axes[1, 1]
    if 'tier_name' in df_with_decisions.columns:
        accepted = df_with_decisions[df_with_decisions['accepted'] == 1]
        accepted['ai_category'] = pd.cut(accepted['ai_percentage'], 
                                         bins=[0, 25, 75, 100], 
                                         labels=['Low AI', 'Medium AI', 'High AI'])
        tier_by_ai = pd.crosstab(accepted['ai_category'], accepted['tier_name'], normalize='index')
        tier_by_ai[['Poster', 'Spotlight', 'Oral']].plot(kind='bar', stacked=True, ax=ax4, 
                                                         color=['steelblue', 'orange', 'green'])
        ax4.set_xlabel('AI Content Category')
        ax4.set_ylabel('Proportion')
        ax4.set_title('Presentation Tier by AI Content')
        ax4.tick_params(axis='x', rotation=0)
        ax4.legend(title='Tier')
    else:
        ax4.text(0.5, 0.5, 'Tier data not available', ha='center', va='center', transform=ax4.transAxes)
        ax4.set_title('Presentation Tier by AI Content')
    
    plt.tight_layout()
    plt.savefig('last_results/acceptance_analysis_plots.png', dpi=150, bbox_inches='tight')
    plt.show()

## 7. Summary Table

In [ ]:
# Create summary table of all tests
if 'accepted' in df_with_decisions.columns:
    summary_rows = []
    
    # Test 1: Probit
    if 'ai_acceptance_probit' in results and results['ai_acceptance_probit']:
        probit = results['ai_acceptance_probit'].get('probit', {})
        if probit:
            summary_rows.append({
                'Test': 'AI → Acceptance (Probit)',
                'Coefficient': probit['params'].get('ai_percentage', np.nan),
                'SE': probit['se'].get('ai_percentage', np.nan),
                'p-value': probit['p_values'].get('ai_percentage', np.nan),
                'Significant': probit['p_values'].get('ai_percentage', 1) < 0.05,
                'N': results['ai_acceptance_probit']['n']
            })
    
    # Test 2: Threshold Discontinuity
    if 'threshold_discontinuity' in results and results['threshold_discontinuity']:
        disc = results['threshold_discontinuity']
        summary_rows.append({
            'Test': 'Threshold Discontinuity',
            'Coefficient': disc['acceptance_difference'],
            'SE': np.nan,
            'p-value': disc.get('chi2', {}).get('p_value', np.nan),
            'Significant': disc.get('chi2', {}).get('p_value', 1) < 0.05,
            'N': disc['n_margin']
        })
    
    # Test 3: Presentation Tier
    if 'presentation_tier' in results and results['presentation_tier']:
        tier = results['presentation_tier'].get('ordered_probit', {})
        if tier:
            summary_rows.append({
                'Test': 'Presentation Tier (Ord. Probit)',
                'Coefficient': tier['params'].get('ai_percentage', np.nan),
                'SE': tier['se'].get('ai_percentage', np.nan),
                'p-value': tier['p_values'].get('ai_percentage', np.nan),
                'Significant': tier['p_values'].get('ai_percentage', 1) < 0.05,
                'N': results['presentation_tier']['n']
            })
    
    # Test 4: Reputation Interaction
    if 'reputation_interaction' in results and results['reputation_interaction']:
        rep = results['reputation_interaction'].get('interaction_model', {})
        if rep:
            summary_rows.append({
                'Test': 'Reputation × AI Interaction',
                'Coefficient': rep['params'].get('ai_x_reputation', np.nan),
                'SE': rep['se'].get('ai_x_reputation', np.nan),
                'p-value': rep['p_values'].get('ai_x_reputation', np.nan),
                'Significant': rep['p_values'].get('ai_x_reputation', 1) < 0.05,
                'N': results['reputation_interaction']['n']
            })
    
    summary_df = pd.DataFrame(summary_rows)
    print("\n" + "=" * 80)
    print("SUMMARY OF ACCEPTANCE ANALYSIS TESTS")
    print("=" * 80)
    print(summary_df.to_string(index=False))
    
    # Save
    summary_df.to_csv('last_results/acceptance_analysis_summary.csv', index=False)
    print("\nSaved to: last_results/acceptance_analysis_summary.csv")